# self参数
self指的是**实例Instance本身**，在`Python`类中规定，函数的第一个参数是实例对象本身，并且约定俗成，把其名字写为`self`，

也就是说，类中的方法的第一个参数一定要是`self`，而且不能省略。关于`self`有三点是很重要的：

+ `self`指的是实例本身，而不是类
+ `self`可以用`this`替代，但是不要这么去写
+ 类的方法中的`self`不可以省略

In [6]:
class Person():
    def eat(self):
        print(self)
        
Alex = Person()
A = Alex.eat()
print(A)
print(Person)
print(Alex)

None
<class '__main__.Person'>


# __ init__ ()方法
在`python`中创建类后，通常会创建一个 `__ init__ ()`方法，这个方法会在创建类的实例的时候自动执行。

`__ init__ ()`方法必须包含一个`self`参数，而且要是第一个参数。

In [9]:
class Person():
     def __init__(self):
         print("小布什")
         #self.name = name
     def eat(self):
         print("钢铁手腕")
            
Alex = Person()

小布什


`__ init__ ()`方法就已经自动执行了，但是如果不是` __ init__ ()`方法，比如说`eat（）`方法，那肯定就只有调用才执行.

再比如说下面的代码，如果` __ init__ ()`方法中还需要传入另一个参数`name`，但是我们在创建`Alex`的实例的时候没有传入`name`，那么程序就会报错:

In [12]:
class Person():
     def __init__(self,name):
         print("小布什")
         self.name = name
     def eat(self):
         print("钢铁手腕")

Bushi_true = Person('小布什')   
print("===+===小===+===布===+===石===+===")
Bushi_false = Person()

小布什
===+===小===+===布===+===石===+===


TypeError: __init__() missing 1 required positional argument: 'name'

我们其实就比较清晰的知道什么东西需要在`__ init__ ()`方法中定义了，就是希望有一些操作是在创建实例的时候就有,

我们写神经网络的代码的时候，一些网络结构的设置，也最好放在`__ init__ ()`方法中。

# super(Net, self).init()
`python`中的`super(Net, self).init()`是指首先找到`Net`的父类（比如是类`NNet`），然后把类`Net`的对象`self`转换为类`NNet`的对象，然后“被转换”的类`NNet`对象调用自己的`init`函数，

其实简单理解就是子类把父类的`__init__()`放到自己的`__init__()`当中，这样子类就有了父类的`__init__()`的那些东西。

`Net`类继承`nn.Module`，`super(Net, self).init()`就是对继承自父类`nn.Module`的属性进行初始化。

而且是用`nn.Module`的初始化方法来初始化继承的属性。

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 输入图像channel：1；输出channel：6；5x5卷积核
        self.conv1 = nn.Conv2d(1, 6, 5)

也就是说，子类继承了父类的所有属性和方法，父类属性自然会用父类方法来进行初始化。
当然，如果初始化的逻辑与父类的不同，不使用父类的方法，自己重新初始化也是可以的。比如：

In [15]:
#!/usr/bin/env python
# -*- coding:utf-8 -*-
 
class Person(object):
    def __init__(self,name,gender,age):
        self.name = name
        self.gender = gender
        self.age = age
 
class Student(Person):
    def __init__(self,name,gender,age,school,score):
        #super(Student,self).__init__(name,gender,age)
        self.name = name.upper()  
        self.gender = gender.upper()
        self.school = school
        self.score = score
 
s = Student('小布石','female',25,'Postgraduate',100)
print(s.school)
print(s.name)
print(s.age) #报错，因为没有初始化

Postgraduate
小布石


AttributeError: 'Student' object has no attribute 'age'

# super的具体用法

In [28]:
class Base(object):
       def __init__(self):
              print("Base init")
 
class Medium1(Base):
       def __init__(self):
              super(Medium1, self).__init__()
              print("Medium1 init")
 
class Medium2(Base):
       def __init__(self):
              super(Medium2, self).__init__()
              print("Medium2 init")
 
class Leaf(Medium1, Medium2):
       def __init__(self):
              super(Leaf, self).__init__()
              print("Leaf init")        
leaf  =Leaf()
Leaf.mro()

Base init
Medium2 init
Medium1 init
Leaf init


[__main__.Leaf, __main__.Medium1, __main__.Medium2, __main__.Base, object]

# super的内核：mro
要理解`super`的原理，就要先了解`mro`。`mro`是`method resolution order`的缩写，表示了类继承体系中的成员解析顺序。

在`python`中，每个类都有一个`mro`的类方法。我们来看一下钻石继承中，`Leaf`类的`mro`是什么样子的：
```python

>>> Leaf.mro()

[__main__.Leaf, __main__.Medium1, __main__.Medium2, __main__.Base, object]
```

可以看到`mro`方法返回的是一个祖先类的列表。`Leaf`的每个祖先都在其中出现一次，这也是`super`在父类中查找成员的顺序。 

通过`mro`，`python`巧妙地将多继承的图结构，转变为`list`的顺序结构。`super`在继承体系中向上的查找过程，变成了在`mro`中向右的线性查找过程，任何类都只会被处理一次。

通过这个方法，`python`解决了多继承中的2大难题：

1. 查找顺序问题。从`Leaf`的`mro`顺序可以看出，如果`Leaf`类通过`super`来访问父类成员，那么`Medium1`的成员会在`Medium2`之前被首先访问到。

    如果`Medium1`和`Medium2`都没有找到，最后再到`Base`中查找。

2. 钻石继承的多次初始化问题。在`mro`的`list`中，`Base`类只出现了一次。事实上任何类都只会在`mro list`中出现一次。

    这就确保了`super`向上调用的过程中，任何祖先类的方法都只会被执行一次。

 

`super(Leaf, self).__init__()`的意思是说：
+ 获取`self`所属类的`mro`, 也就是`[Leaf, Medium1, Medium2, Base]`.
+ 从`mro`中`Leaf`右边的一个类开始，依次寻找`__init__`函数。这里是从`Medium1`开始寻找.
+ 一旦找到，就把找到的`__init__`函数绑定到`self`对象，并返回.

从这个执行流程可以看到，如果我们不想调用`Medium1`的`__init__`，而想要调用`Medium2`的`__init__`，那么`super`应该写成：`super(Medium1, self)__init__()`

In [30]:
class Leaf(Medium1, Medium2):
    def __new__(cls):
        obj = super(Leaf, cls).__new__(cls)
        print("Leaf new")
        return obj

`super(Leaf, cls).__new__(cls)`的意思是说：

+ 获取`cls`这个类的`mro`，这里也是`[Leaf, Medium1, Medium2, Base]`
+ 从`mro`中`Leaf`右边的一个类开始，依次寻找`__new__`函数
+ 一旦找到，就返回“非绑定”的`__new__`函数

由于返回的是非绑定的函数对象，因此调用时不能省略函数的第一个参数。这也是这里调用`__new__`时，需要传入参数`cls`的原因

同样的，如果我们想从某个`mro`的某个位置开始查找，只需要修改`super`的第一个参数就行